In [1]:
import gurobi
import pandas as pd

### parameter erstellen

In [2]:
#individual weight pw pw[1] für pojekt 1 pw[2]für Projekt 2 un
#pw1 für Zeit weight,pw2 für Resource weight 
pw=[(0.9,0.1),(0.8,0.2)]


In [3]:
#Zeit und Resource Bedarf erstellen
#zwei Projekt mit jeder 7 Aufgabe

In [4]:
time=[[3,2,2,5,2],[1,3,5,2,2]]
resource=[[4,1,2,3,1],[4,2,1,1,2]]

In [286]:
time

[[3, 2, 2, 5, 2], [1, 3, 5, 2, 2]]

### Graph erstellen

In [363]:
G1=pd.DataFrame([0]*5 for _ in range(5))
G2=pd.DataFrame([0]*5 for _ in range(5))

In [364]:
G1.loc[0,1]=1
G1.loc[0,2]=1
G1.loc[0,3]=1
G1.loc[3,4]=1

G2.loc[0,2]=1
G2.loc[1,3]=1
G2.loc[1,4]=1


In [365]:
depec1=G1.apply(sum)
depec2=G2.apply(sum)
depencies=[[],[]]
for i in range(5):
    depencies[0].append(depec1[i])
    depencies[1].append(depec2[i])
    
    

In [307]:
###这是入度，入度为0表示接下来可以做的任务
depencies

[[0, 1, 1, 1, 1], [0, 0, 1, 1, 1]]

In [308]:
G2.apply(sum)

0    0
1    0
2    1
3    1
4    1
dtype: int64

In [309]:
G1.loc[0]


0    0
1    1
2    1
3    1
4    0
Name: 0, dtype: int64

In [354]:
G2

,0,1,2,3,4
0,0,0,1,0,0
1,0,0,0,1,1
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [355]:
G1

,0,1,2,3,4
0,0,1,1,1,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,1
4,0,0,0,0,0


In [356]:
Jobqueue=[]
#######计算入度###########
depec1=G1.apply(sum)
depec2=G2.apply(sum)
depencies=[[],[]]
for i in range(5):
    depencies[0].append(depec1[i])
    depencies[1].append(depec2[i])
##########把入度为0且需要资源小于等于可用资源的Job添加到Jobqueue里###########
for i in range(len(depencies)):
    for j in range(len(depencices[i])):
        if depencices[i][j]==0 and Resource-resource[i][j]>=0:
            Jobqueue.append([i,j])
    

In [341]:
Jobqueue

[]

### Simulator

In [2]:
pw=[(0.9,0.1),(0.8,0.2)]
time=[[3,2,2,5,2],[1,3,5,2,2]]
resource=[[4,1,2,3,1],[4,2,1,1,2]]
G1=pd.DataFrame([0]*5 for _ in range(5))
G2=pd.DataFrame([0]*5 for _ in range(5))
G1.loc[0,1]=1
G1.loc[0,2]=1
G1.loc[0,3]=1
G1.loc[3,4]=1

G2.loc[0,2]=1
G2.loc[1,3]=1
G2.loc[1,4]=1

Total_Resource=4
remain_Resource=4
t=0
job_started=[]
job_finished=[]
time_finished=[]
doing_job=[]
job_end_time=[]
finished_Project=False
while finished_Project==False:
    
#for n in range(30):
    executable_jobs=[]

    #######计算入度###########
    depec1=G1.apply(sum)
    depec2=G2.apply(sum)
    ##########把入度为0且需要资源小于等于可用资源的Job添加到executable_jobs里###########
    for i in range(len(depec1)):
        if depec1[depec1.index[i]]==0 and remain_Resource-resource[0][depec1.index[i]]>=0:
            executable_jobs.append([0,depec1.index[i]])
    for j in range(len(depec2)):
        if depec2[depec2.index[j]]==0 and remain_Resource-resource[1][depec2.index[j]]>=0:
            executable_jobs.append([1,depec2.index[j]]) 
   
     #######检测正在做的任务是否添加进了executable_jobs######
    if doing_job != 0:
        for i in range(len(executable_jobs)):
            for j in executable_jobs:
                if j in doing_job:
                    executable_jobs.remove(j)
        ##
        #copy=executable_jobs[:]
        #for i in range(len(doing_job)):
            #for j in range(len(copy)):
                #if doing_job[i]==copy[j]:
                    #executable_jobs.pop(j)
    print('t={}s时刻,可用资源为{}，正在执行的任务有{},可以执行的任务有{},已经完成的任务有{}'.format(t,remain_Resource,doing_job,executable_jobs,job_finished))
    ###########如果executable_jobs中有任务，开始计算他们的优先级#############
    if len(executable_jobs)!=0:
        priority=[]
        time_queue=[]
        resource_queue=[]
        for i in range(len(executable_jobs)):
            time_queue.append(time[executable_jobs[i][0]][executable_jobs[i][1]])
            resource_queue.append(resource[executable_jobs[i][0]][executable_jobs[i][1]])
            
        max_time_in_queue=max(time_queue)        
        max_resource_in_queue=max(resource_queue)
        #print('可执行任务中，消耗最多得时间是{},消耗最多的资源是{}'.format(max_time_in_queue,max_resource_in_queue))
         ###########计算优先度############
        for i in range(len(executable_jobs)):
            priority.append(pw[executable_jobs[i][0]][1]*(time_queue[i]/max_time_in_queue)+pw[executable_jobs[i][0]][0]*(resource_queue[i]/max_resource_in_queue))
        max_priotity=max(priority)
        for i in range(len(priority)):
            if priority[i] == max_priotity:
                todo_job=executable_jobs[i]
        doing_job.append(todo_job)
        
        #print('可执行任务中，他们的优先级分别为{},最大是的{},执行{}任务,正在执行的任务有{}'.format(priority,max_priotity,todo_job,doing_job))
        ###########计算消耗的资源和每个正在做的任务预计完成的时间####################
        remain_Resource-=resource[todo_job[0]][todo_job[1]]
        job_end_time.append(t+time[todo_job[0]][todo_job[1]])
        #############为完成时间和正在进行的任务排序，方便接下来弹出################
        
        
        ####################################################################
        #print('可用资源为{},正在执行的任务的预计完成时间为{}'.format(remain_Resource,job_end_time))
    ###########如果executable_jobs中没有任务，则说明此时刻已经没有可以完成的任务了，跳到t=t+1时刻#############   
    else:
        t+=1
        if len(job_finished)==10 and remain_Resource==4:
            finished_Project=True
        if len(job_end_time)==1:
            if t==job_end_time[0]:
                remain_Resource=remain_Resource+resource[doing_job[0][0]][doing_job[0][1]]
                if doing_job[0][0]==0:
                    G1=G1.drop(doing_job[0][1]).drop(columns=doing_job[0][1])
                else:
                    G2=G2.drop(doing_job[0][1]).drop(columns=doing_job[0][1])
                job_finished.append(doing_job.pop())
                time_finished.append(job_end_time.pop())
        elif len(job_end_time) > 1: 
            copy_time=job_end_time[:]
            copy_resource=doing_job[:]
            for i in range(len(copy_time)):
                if t == copy_time[i]:
                    remain_Resource=remain_Resource+resource[copy_resource[i][0]][copy_resource[i][1]]
                    if copy_resource[i][0]==0:
                        G1=G1.drop(copy_resource[i][1]).drop(columns=copy_resource[i][1])
                    else:
                        G2=G2.drop(copy_resource[i][1]).drop(columns=copy_resource[i][1])
                    
            lst=[]
            for i in range(len(job_end_time)):
                if t == job_end_time[i]:
                    lst.append(i)
            lst.sort(reverse=True)
            for i in range(len(lst)):
                
                job_finished.append(doing_job.pop(lst[i]))
                time_finished.append(job_end_time.pop(lst[i]))

print('完成的任务分别为{},它们完成的时间分别为{}'.format(job_finished,time_finished))
project1_checked=False
project2_checked=False
for i in range(len(job_finished)-1,-1,-1):
    if job_finished[i][0]==1 and project2_checked==False:
        project2_finished_time=time_finished[i]
        project2_checked =True
    elif job_finished[i][0]==0 and project1_checked == False:
        project1_finished_time=time_finished[i]
        project1_checked = True
print('两任务完成的时间分别为{},{}'.format(project1_finished_time,project2_finished_time))

Delay=project1_finished_time-13+project2_finished_time-15
print('任务延迟为{}'.format(Delay))

t=0s时刻,可用资源为4，正在执行的任务有[],可以执行的任务有[[0, 0], [1, 0], [1, 1]],已经完成的任务有[]
t=0s时刻,可用资源为0，正在执行的任务有[[0, 0]],可以执行的任务有[],已经完成的任务有[]
t=1s时刻,可用资源为0，正在执行的任务有[[0, 0]],可以执行的任务有[],已经完成的任务有[]
t=2s时刻,可用资源为0，正在执行的任务有[[0, 0]],可以执行的任务有[],已经完成的任务有[]
t=3s时刻,可用资源为4，正在执行的任务有[],可以执行的任务有[[0, 1], [0, 2], [0, 3], [1, 0], [1, 1]],已经完成的任务有[[0, 0]]
t=3s时刻,可用资源为0，正在执行的任务有[[1, 0]],可以执行的任务有[],已经完成的任务有[[0, 0]]
t=4s时刻,可用资源为4，正在执行的任务有[],可以执行的任务有[[0, 1], [0, 2], [0, 3], [1, 1], [1, 2]],已经完成的任务有[[0, 0], [1, 0]]
t=4s时刻,可用资源为1，正在执行的任务有[[0, 3]],可以执行的任务有[[0, 1], [1, 2]],已经完成的任务有[[0, 0], [1, 0]]
t=4s时刻,可用资源为0，正在执行的任务有[[0, 3], [1, 2]],可以执行的任务有[],已经完成的任务有[[0, 0], [1, 0]]
t=5s时刻,可用资源为0，正在执行的任务有[[0, 3], [1, 2]],可以执行的任务有[],已经完成的任务有[[0, 0], [1, 0]]
t=6s时刻,可用资源为0，正在执行的任务有[[0, 3], [1, 2]],可以执行的任务有[],已经完成的任务有[[0, 0], [1, 0]]
t=7s时刻,可用资源为0，正在执行的任务有[[0, 3], [1, 2]],可以执行的任务有[],已经完成的任务有[[0, 0], [1, 0]]
t=8s时刻,可用资源为0，正在执行的任务有[[0, 3], [1, 2]],可以执行的任务有[],已经完成的任务有[[0, 0], [1, 0]]
t=9s时刻,可用资源为4，正在执行的任务有[],可以执行的任务有[[0, 1], [0, 2], [0, 4], [1, 1]],已

In [215]:
from gurobipy import *
import pandas as pd
def Simulator(pw:list):
    time=[[3,2,2,5,2],[1,3,5,2,2]]
    resource=[[4,1,2,3,1],[4,2,1,1,2]]
    G1=pd.DataFrame([0]*5 for _ in range(5))
    G2=pd.DataFrame([0]*5 for _ in range(5))
    G1.loc[0,1]=1
    G1.loc[0,2]=1
    G1.loc[0,3]=1
    G1.loc[3,4]=1
    G2.loc[0,2]=1
    G2.loc[1,3]=1
    G2.loc[1,4]=1
    Total_Resource=4
    remain_Resource=4
    t=0
    job_started=[]
    job_finished=[]
    time_finished=[]
    doing_job=[]
    job_end_time=[]
    finished_Project=False
    while finished_Project==False:
        executable_jobs=[]
        #######计算入度###########
        depec1=G1.apply(sum)
        depec2=G2.apply(sum)
        ##########把入度为0且需要资源小于等于可用资源的Job添加到executable_jobs里###########
        for i in range(len(depec1)):
            if depec1[depec1.index[i]]==0 and remain_Resource-resource[0][depec1.index[i]]>=0:
                executable_jobs.append([0,depec1.index[i]])
        for j in range(len(depec2)):
            if depec2[depec2.index[j]]==0 and remain_Resource-resource[1][depec2.index[j]]>=0:
                executable_jobs.append([1,depec2.index[j]]) 
         #######检测正在做的任务是否添加进了executable_jobs######
        if doing_job != 0:
            for i in range(len(executable_jobs)):
                for j in executable_jobs:
                    if j in doing_job:
                        executable_jobs.remove(j)
        #print('t={}s时刻,可用资源为{}，正在执行的任务有{},可以执行的任务有{},已经完成的任务有{}'.format(t,remain_Resource,doing_job,executable_jobs,job_finished))
        ###########如果executable_jobs中有任务，开始计算他们的优先级#############
        if len(executable_jobs)!=0:
            priority=[]
            time_queue=[]
            resource_queue=[]
            for i in range(len(executable_jobs)):
                time_queue.append(time[executable_jobs[i][0]][executable_jobs[i][1]])
                resource_queue.append(resource[executable_jobs[i][0]][executable_jobs[i][1]])
            max_time_in_queue=max(time_queue)        
            max_resource_in_queue=max(resource_queue)
            #print('可执行任务中，消耗最多得时间是{},消耗最多的资源是{}'.format(max_time_in_queue,max_resource_in_queue))
            ###########计算优先度############
            for i in range(len(executable_jobs)):
                priority.append(pw[executable_jobs[i][0]][1]*(time_queue[i]/max_time_in_queue)+pw[executable_jobs[i][0]][0]*(resource_queue[i]/max_resource_in_queue))
            #max_priotity=priority[0]
            max_priotity=max(priority)
            for i in range(len(priority)):
                if priority[i] == max_priotity:
                    todo_job=executable_jobs[i]
            doing_job.append(todo_job)
            ###########计算消耗的资源和每个正在做的任务预计完成的时间####################
            remain_Resource-=resource[todo_job[0]][todo_job[1]]
            job_end_time.append(t+time[todo_job[0]][todo_job[1]])
            ####################################################################
            #print('可用资源为{},正在执行的任务的预计完成时间为{}'.format(remain_Resource,job_end_time))
            ###########如果executable_jobs中没有任务，则说明此时刻已经没有可以完成的任务了，跳到t=t+1时刻#############   
        else:
            t+=1
            if len(job_finished)==10 and remain_Resource==4:
                finished_Project=True
            if len(job_end_time)==1:
                if t==job_end_time[0]:
                    remain_Resource=remain_Resource+resource[doing_job[0][0]][doing_job[0][1]]
                    if doing_job[0][0]==0:
                        G1=G1.drop(doing_job[0][1]).drop(columns=doing_job[0][1])
                    else:
                        G2=G2.drop(doing_job[0][1]).drop(columns=doing_job[0][1])
                    job_finished.append(doing_job.pop())
                    time_finished.append(job_end_time.pop())
            elif len(job_end_time) > 1: 
                copy_time=job_end_time[:]
                copy_resource=doing_job[:]
                for i in range(len(copy_time)):
                    if t == copy_time[i]:
                        remain_Resource=remain_Resource+resource[copy_resource[i][0]][copy_resource[i][1]]
                        if copy_resource[i][0]==0:
                            G1=G1.drop(copy_resource[i][1]).drop(columns=copy_resource[i][1])
                        else:
                            G2=G2.drop(copy_resource[i][1]).drop(columns=copy_resource[i][1])

                lst=[]
                for i in range(len(job_end_time)):
                    if t == job_end_time[i]:
                        lst.append(i)
                lst.sort(reverse=True)
                for i in range(len(lst)):

                    job_finished.append(doing_job.pop(lst[i]))
                    time_finished.append(job_end_time.pop(lst[i]))

        #print('完成的任务分别为{},它们完成的时间分别为{}'.format(job_finished,time_finished))
    project1_checked=False
    project2_checked=False
    for i in range(len(job_finished)-1,-1,-1):
        if job_finished[i][0]==1 and project2_checked==False:
            project2_finished_time=time_finished[i]
            project2_checked =True
        elif job_finished[i][0]==0 and project1_checked == False:
            project1_finished_time=time_finished[i]
            project1_checked = True
        #print('两任务完成的时间分别为{},{}'.format(project1_finished_time,project2_finished_time))

    delay=project1_finished_time-13+project2_finished_time-15

    return delay

In [216]:
Simulator([(0.9,0.1),(0.8,0.2)])

0

In [206]:
m=Model()

In [217]:
pw1=m.addVar(0,1,name='pw1')
pw2=m.addVar(0,1,name='pw2')
pw3=m.addVar(0,1,name='pw3')
pw4=m.addVar(0,1,name='pw4')

In [218]:
pw=[(pw1,pw2),(pw3,pw4)]

In [219]:
type(pw1)

gurobipy.Var

In [220]:
m.setObjective(Simulator(pw),GRB.MINIMIZE)

NotImplementedError: 

In [221]:
c1=m.addConstr(pw1+pw2==1)
c2=m.addConstr(pw3+pw4==1)

In [222]:
m

<gurobi.Model Continuous instance Unnamed: 0 constrs, 0 vars, No parameter changes>

In [44]:
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 2 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xa9b0eae3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00


In [45]:
m.printAttr

<bound method Model.printAttr of <gurobi.Model Continuous instance Unnamed: 2 constrs, 4 vars, No parameter changes>>

In [190]:
m = Model()
m

<gurobi.Model Continuous instance Unnamed: 0 constrs, 0 vars, No parameter changes>

In [191]:
x = m.addVar(0,1,name="x")
y = m.addVar(0,1,name="y")
z = m.addVar(0,1,name="z")

In [195]:
def X(T:list):
    a=T[0]
    return T[0] + T[1] + 2*T[2]+a

In [203]:
type(x)

gurobipy.Var

In [197]:
X([x,y,z])

AttributeError: Index out of range for attribute 'VarName'

In [198]:
m.setObjective(X([x,y,z]), GRB.MINIMIZE)

In [199]:
c1 = m.addConstr(x + 2*y + 3*z <= 4)
c2 = m.addConstr(x + y >= 1)

In [200]:
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xc6101cf7
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   1.000000e+00      0s
Extra one simplex iteration after uncrush
       1    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.000000000e+00


In [201]:
m.printAttr('X')


    Variable            X 
-------------------------
           y            1 


In [202]:
m.ObjVal

1.0

In [1]:
from gurobipy import *

In [6]:
Processing_time={'1_1':0,
                 '2_1':3,
                 '3_1':2,
                 '4_1':2,
                 '5_1':5,
                 '6_1':2,
                 '7_1':0,
                 '1_2':0,
                 '2_2':1,
                 '3_2':3,
                 '4_2':5,
                 '5_2':2,
                 '6_2':2,
                 '7_2':0
    
}
Resource_used={'1_1':0,
                 '2_1':4,
                 '3_1':1,
                 '4_1':2,
                 '5_1':3,
                 '6_1':1,
                 '7_1':0,
                 '1_2':0,
                 '2_2':4,
                 '3_2':2,
                 '4_2':1,
                 '5_2':1,
                 '6_2':2,
                 '7_2':0
    
}
Job1s=['1_1','2_1','3_1','4_1','5_1','6_1','7_1']
Job2s=['1_2','2_2','3_2','4_2','5_2','6_2','7_2']
Jobs=Job1s+Job2s
times=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
s1=0
s2=0
R=4
T=20

In [7]:
# lst=list(Processing_time.values())

time

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [8]:
m=Model()
x=m.addVars(Jobs,time,name='start_time_bool',vtype=GRB.BINARY)
omiga=m.addVars(Jobs,lb=0,ub=T,name='end_time',vtype=GRB.INTEGER)
alpha=m.addVars(Jobs,lb=0,ub=T,name='start_time',vtype=GRB.INTEGER)
y=m.addVars(Jobs,time,name='dauer_time_bool',vtype=GRB.BINARY)
#Verspätung=m.addVar(vtype=GRB.INTEGER,name='Verspätung')

In [9]:
m.addConstrs((alpha[Job]==omiga[Job]-Processing_time[Job] for Job in Jobs))
m.addConstrs(alpha[Job]>=s1 for Job in Job1s)
m.addConstrs(alpha[Job]>=s2 for Job in Job2s)

m.addConstr(alpha['2_1']>=omiga['1_1'])
m.addConstr(alpha['3_1']>=omiga['2_1'])
m.addConstr(alpha['4_1']>=omiga['2_1'])
m.addConstr(alpha['5_1']>=omiga['2_1'])
m.addConstr(alpha['6_1']>=omiga['5_1'])
m.addConstr(alpha['7_1']>=omiga['3_1'])
m.addConstr(alpha['7_1']>=omiga['4_1'])
m.addConstr(alpha['7_1']>=omiga['6_1'])
m.addConstr(alpha['2_2']>=omiga['1_2'])
m.addConstr(alpha['3_2']>=omiga['1_2'])
m.addConstr(alpha['4_2']>=omiga['2_2'])
m.addConstr(alpha['5_2']>=omiga['3_2'])
m.addConstr(alpha['6_2']>=omiga['3_2'])
m.addConstr(alpha['7_2']>=omiga['4_2'])
m.addConstr(alpha['7_2']>=omiga['5_2'])
m.addConstr(alpha['7_2']>=omiga['6_2'])

for job in Jobs:
    m.addConstr(quicksum(x[(job),time] for time in times)==1)


for job in Jobs:
    m.addConstr(quicksum(y[(job),time] for time in times)==Processing_time[job])



for job in Jobs:
    for timee in time[0:16]:
        m.addGenConstrIndicator(x[job,timee],True,quicksum(y[job,timee+timeee] for timeee in range(Processing_time[job])),GRB.EQUAL,Processing_time[job])
        


for i in range(len(times)):
    m.addConstr(quicksum(y[(Job),times[i]]*Resource_used[Job] for Job in Jobs)<=R)
    
for job in Jobs:
    m.addConstr(quicksum(x[(job),time]*time for time in times)==alpha[job])

    
#m.addGenConstrAbs(Verspätung,omiga['7_1']-13+omiga['7_2']-15)
    

In [10]:
m.setObjective(omiga['7_1']-13+omiga['7_2']-15, GRB.MINIMIZE)


In [11]:
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 106 rows, 588 columns and 1128 nonzeros
Model fingerprint: 0xbdf75438
Model has 224 general constraints
Variable types: 0 continuous, 588 integer (560 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 5e+00]
Presolve added 73 rows and 0 columns
Presolve removed 0 rows and 255 columns
Presolve time: 0.02s
Presolved: 179 rows, 333 columns, 1119 nonzeros
Variable types: 0 continuous, 333 integer (320 binary)
Found heuristic solution: objective 12.0000000

Root relaxation: objective -9.000000e+00, 305 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -9.00000    0   55   12.00000   -9.00000   175%     -    0s
     0     0   -7.36000    0   88   12.00000   -7.36000  

In [12]:
m.printAttr('X')


    Variable            X 
-------------------------
start_time_bool[1_1,1]            1 
start_time_bool[2_1,7]            1 
start_time_bool[3_1,12]            1 
start_time_bool[4_1,15]            1 
start_time_bool[5_1,10]            1 
start_time_bool[6_1,15]            1 
start_time_bool[7_1,17]            1 
start_time_bool[1_2,1]            1 
start_time_bool[2_2,1]            1 
start_time_bool[3_2,2]            1 
start_time_bool[4_2,2]            1 
start_time_bool[5_2,5]            1 
start_time_bool[6_2,5]            1 
start_time_bool[7_2,7]            1 
end_time[1_1]            1 
end_time[2_1]           10 
end_time[3_1]           14 
end_time[4_1]           17 
end_time[5_1]           15 
end_time[6_1]           17 
end_time[7_1]           17 
end_time[1_2]            1 
end_time[2_2]            2 
end_time[3_2]            5 
end_time[4_2]            7 
end_time[5_2]            7 
end_time[6_2]            7 
end_time[7_2]            7 
start_time[1_1]            1 
s

In [13]:
m.ObjVal

-4.000000000000007

In [22]:
omiga['4_2'].X

7.0

In [395]:
m.write('Verspätung.lp')

GurobiError: Can't import file 'Verspätung.lp' into a model